In [4]:
import cv2
import numpy as np

net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
classes = []

In [5]:
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] -1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [6]:
class KF(object):
    def __init__(self,dt,acc_x,acc_y,sigma_x,sigma_y,sigma_acc):
        
        self.dt = dt
        self.acc = np.array([[acc_x],[acc_y]])

        # initialize the first state
        self.X = np.zeros([4,1])
        #initialize the covariance matrix by guessing a value. The guess can improve the accuracy of the initial predicted state
        self.P = np.eye(4)
        
        #Now we need to define the matrics F, B, and H in order to have everything we need to the 2 Kalman filter equations
        self.B = np.array([[0.5*(self.dt**2), 0],[0.5*(self.dt**2),0],[self.dt,0],[0,self.dt]])
        self.F = np.array([[1, 0, self.dt, 0],[0, 1, 0, self.dt],[0, 0, 1, 0],[0, 0, 0, 1]])
        self.H = np.array([[1, 0, 0, 0],[0, 1, 0, 0]])

        #Later when we try to update the state, we will need the covariance matrics for the noises in the measurements and process
        self.R = np.array([[sigma_x**2,0],[0,sigma_y**2]])
        self.Q = np.array([[0.25*(self.dt**4), 0, 0.5*(self.dt**3), 0],[0,0.25*(self.dt**4), 0,0.5*(self.dt**3)],[0.5*(self.dt**3), 0, self.dt**2, 0],[0, 0.5*(self.dt**3), 0, self.dt**2]])*sigma_acc**2

    def predict(self):
        self.X = np.dot(self.F,self.X)+np.dot(self.B,self.acc)
        self.P = np.dot(np.dot(self.F,self.P),self.F.T)+self.Q
        return self.X

    def update(self, z):
        S = np.dot(np.dot(self.H,self.P),self.H.T)+self.R
        K = np.dot(np.dot(self.P,self.H.T),np.linalg.inv(S))
        self.P = np.dot((np.eye(4)-np.dot(K,self.H)),self.P)
        self.X = self.X + np.dot(K, (z - np.dot(self.H, self.X)))
        return self.X
    
    
KalmanFilter = KF(0.5, 2, 3, 0.2, 0.3,0.5)

In [10]:
   #Load Image
vid = cv2.VideoCapture("obj.mp4")
i = 0
while(1):
    ret, img = vid.read()
    i = i+1 
    if i == 15:
        continue
    i = 0
    img = cv2.resize(img, None, fx=0.4, fy =0.4)
    frame = img
    height, width, channels = img.shape
    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(outputlayers)
    blurred = cv2.GaussianBlur(frame,(5,5),0)
    canny = cv2.Canny(blurred,  50, 190, 3)
    _, img_thresh = cv2.threshold(canny, 254, 255,cv2.THRESH_BINARY)
    
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    centroids = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                (x, y) = center_x, center_y
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                centroids.append(np.array([[int(x)], [int(y)]]))
        if (len(centroids) > 0):
            cv2.circle(frame, (int(centroids[0][0]), int(centroids[0][1])), 10, (0, 255, 0), 2)
            State_predicted = KalmanFilter.predict()
            (m,n) = State_predicted[0:2] # we only need the position values which are the first 2 elements in the X_k array
            cv2.circle(frame, (m+3,n+3), 10, (0, 0, 255), 2)
            print(centroids[0])
            State_updated = KalmanFilter.update(centroids[0])
            (a,b) = State_updated[0:2]
            cv2.circle(frame, (a+6,b+6), 10, (255, 0, 0), 2)
            cv2.putText(frame, "position measured", (centroids[0][0], centroids[0][1]), 0, 0.5, (0,255,0), 2)
            cv2.putText(frame, "position predicted", (m+3, n+3), 0, 0.5, (0,0,255), 2)
            cv2.putText(frame, "position updated", (a+6,b+6), 0, 0.5, (255,0,0), 2)
     # Non-Max Supression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 3)


    cv2.imshow("image",frame + img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            vid.release()
            cv2.destroyAllWindows()
            break

[[208]
 [  3]]
[[208]
 [  3]]
[[208]
 [  3]]
[[0]
 [4]
 [1]
 [5]
 [6]
 [3]]
[[209]
 [  3]]
[[209]
 [  3]]
[[209]
 [  3]]
[[0]
 [3]
 [4]
 [1]
 [5]
 [2]]
[[206]
 [  3]]
[[206]
 [  3]]
[[206]
 [  3]]
[[0]
 [3]
 [5]
 [4]
 [1]]
[[208]
 [  1]]
[[208]
 [  1]]
[[208]
 [  1]]
[[0]
 [2]
 [5]
 [3]
 [4]]
[[208]
 [  1]]
[[208]
 [  1]]
[[208]
 [  1]]
[[0]
 [2]
 [6]
 [4]
 [1]
 [5]
 [3]]
[[209]
 [  3]]
[[209]
 [  3]]
[[209]
 [  3]]
[[0]
 [2]
 [3]
 [5]
 [1]
 [6]]
[[209]
 [  3]]
[[209]
 [  3]]
[[209]
 [  3]]
[[0]
 [2]
 [4]
 [1]
 [5]
 [7]
 [3]]
[[209]
 [  1]]
[[209]
 [  1]]
[[209]
 [  1]]
[[2]
 [0]
 [4]
 [6]
 [5]
 [1]
 [3]
 [7]]
[[209]
 [  1]]
[[209]
 [  1]]
[[209]
 [  1]]
[[0]
 [6]
 [2]
 [5]
 [4]
 [1]
 [3]]
[[211]
 [  3]]
[[211]
 [  3]]
[[211]
 [  3]]
[[0]
 [5]
 [1]
 [3]
 [4]
 [6]]
[[211]
 [  5]]
[[211]
 [  5]]
[[211]
 [  5]]
[[8]
 [3]
 [0]
 [6]
 [7]
 [2]
 [1]
 [5]]
[[211]
 [  2]]
[[211]
 [  2]]
[[211]
 [  2]]
[[0]
 [3]
 [8]
 [7]
 [6]
 [1]
 [2]
 [5]]
[[211]
 [  4]]
[[211]
 [  4]]
[[211]
 [  4]]
[[0]
 [2